In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset

/opt/homebrew/Caskroom/miniconda/base/envs/vcnlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 1. 데이터 준비하기
def prepare_dataset(file_path):
    """
    file_path: 데이터를 포함한 CSV 파일 경로
    파일은 'input'과 'output' 열을 포함해야 합니다.
    """
    import pandas as pd
    data = pd.read_csv(file_path)
    dataset = Dataset.from_pandas(data)
    return dataset

# 사용자 정의 데이터 경로
train_file = "train.csv"  # 학습 데이터 경로
valid_file = "valid.csv"  # 검증 데이터 경로

dataset = DatasetDict({
    "train": prepare_dataset(train_file),
    "validation": prepare_dataset(valid_file)
})

# 2. 토크나이저 및 모델 로드
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# 3. 데이터셋 전처리
def preprocess_function(examples):
    """
    데이터를 T5 입력 형식으로 변환
    """
    inputs = examples["input"]  # 'input' 열 사용
    targets = examples["output"]  # 'output' 열 사용
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels

    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 데이터셋 셔플링
tokenized_datasets["train"] = tokenized_datasets["train"].shuffle(seed=42)

# 4. 학습 설정
training_args = TrainingArguments(
    output_dir="./results",  # 출력 디렉토리
    evaluation_strategy="epoch",  # 평가 주기
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,  # 체크포인트 제한
    save_strategy="epoch",
    logging_dir="./logs",  # 로깅 디렉토리
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="loss",  # 검증 기준
)

# 5. 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# 6. 학습 시작
trainer.train()

# 7. 모델 저장
trainer.save_model("./t5-custom-model")
tokenizer.save_pretrained("./t5-custom-model")
